In [ ]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import itertools
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
# Apparently, SNS stands for "Samuel Norman Seaborn", a fictional
# character from The West Wing
import seaborn as sns
import sympy

sns.set()
sympy.init_printing()
# Make the figures directory if it doesn't exist.
Path('figures/').mkdir(exist_ok=True)

In [ ]:
def generate_individual(n, scale=100):
    """Generates an individual solution of the given size.

    Pick random coordinates in the `scale`x`scale` grid uniformly.

    :param n: The size of the individual to generate.
    :param scale: How much to scale the individual's coordinates by.
    """
    # Scale up the uniform values from [0, 1].
    return np.random.rand(n, 2) * scale

def generate_population(size, n, scale=100):
    """Generate a population of individuals with the given size.

    Pick random coordinates in the `scale`x`scale` grid uniformly.

    :param size: The number of individuals to generate.
    :param n: The size of each individual.
    :param scale: How much to scale the individual's coordinates by.
    """
    # Scale up the uniform values from [0, 1].
    return np.random.rand(size, n, 2) * scale

In [ ]:
PROBLEM_SIZE = 50
individual = generate_individual(PROBLEM_SIZE)
plt.plot(individual[:, 0], individual[:, 1], 'r')
plt.plot(individual[:, 0], individual[:, 1], 'o')
plt.title('A random individual')
# plt.axis('equal')
plt.axis('scaled')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.savefig('figures/prob2-random-individual.pdf')
plt.show()

In [ ]:
def pairwise(iterable):
    """Iterate over the given iterable in pairs.

    pairwise([1, 2, 3, 4]) -> (1, 2), (2, 3), (3, 4)
    """
    a, b = itertools.tee(iterable)
    # Advance b one step
    next(b, None)
    return zip(a, b)

In [ ]:
def fitness(individual):
    """Evaluate the fitness of the given individual.

    Compute the Euclidean distance between every pair of cities in the individual
    and add them together.

    :param individual: An array of cities, where each city is an (x, y) pair.
    :type individual: np.ndarray with shape (n, 2)
    :return: The fitness of the individual.
    """
    return sum(np.linalg.norm(c1 - c2) for c1, c2 in pairwise(individual))

In [ ]:
def swap_mutation(individual):
    """Swap two random cities in the individual.

    Returns a new mutated copy of the given array.
    """
    # Arrays are (kind of) passed by reference in Python.
    x = np.copy(individual)
    # Generate two valid indices to swap.
    i = np.random.randint(0, len(x) - 1)
    j = np.random.randint(0, len(x) - 1)

    x[i], x[j] = x[j], x[i]

    return x

In [ ]:
a = np.array(range(10))
swap_mutation(a)

In [ ]:
def insertion_mutation(individual):
    """Insert a random value in the list somewhere else in the list.

    Returns a new mutated copy of the given array.
    """
    i = np.random.randint(0, len(individual) - 1)
    j = np.random.randint(0, len(individual) - 1)

    # Delete the element at index j and insert it before index i.
    return np.insert(np.delete(individual, j), i, individual[j])

In [ ]:
insertion_mutation(a)

In [ ]:
def displacement_mutation(individual):
    """Inserts a random subarray in the list somewhere else.

    Returns a new mutated copy of the given array.
    """
    # Pick a random subarray
    i = np.random.randint(0, len(individual) - 2)
    j = np.random.randint(i, len(individual) - 1)
    subarray = individual[i:j]
    # Delete the given subarray
    tmp = np.delete(individual, range(i, j))
    k = np.random.randint(0, len(tmp) - 1)

    return np.insert(tmp, k, subarray)

In [ ]:
displacement_mutation(a)

In [ ]:
def shuffle_mutation(individual):
    """Shuffles a random subarray in the given individual.

    Returns a new mutated copy of the given array.
    """
    # Pick a random subarray
    i = np.random.randint(0, len(individual) - 2)
    j = np.random.randint(i, len(individual) - 1)
    x = np.copy(individual)
    np.random.shuffle(x[i:j])

    return x

In [ ]:
shuffle_mutation(a)

In [ ]:
def inversion_mutation(individual):
    """Inverts a random subarray in the given individual.

    Returns a new mutated copy of the given array.
    """
    i = np.random.randint(0, len(individual) - 2)
    j = np.random.randint(i, len(individual) - 1)
    x = np.copy(individual)
    # Invert the subarray.
    x[i:j] = x[i:j][::-1]

    return x

In [ ]:
inversion_mutation(a)

In [ ]:
def mutate(individual, method='inversion'):
    """Mutate the given individual via the given method.

    Returns a new mutated copy of the given array.

    :param method: One of 'swap', 'insertion', 'displacement',
    'shuffle', or 'inversion'. Defaults to 'inversion'.
    """
    methods = {
        'swap': swap_mutation,
        'insertion': insertion_mutation,
        'displacement': displacement_mutation,
        'shuffle': shuffle_mutation,
        'inversion': inversion_mutation,
    }
    return methods[method](individual)

In [ ]:
mutate(a)